In [34]:
folder = "./Data/bybit"
folder_lab1 = "./Data/lab1"
file = "ETH-USDT:USDT_1m.csv"
periods = [5, 10, 15, 20, 30, 60, 120]

In [42]:
folder = "./Data/gemini"
folder_lab1 = "./Data/lab1"
file = "gemini_1m_features.csv"
period = 10
periods = [period]


In [43]:
import pandas as pd


features_base = ['timestamp', 'close']
features_stochrsi = ['stoch_rsi_overbought', 'stoch_rsi_oversold', 'stoch_rsi_bullish_crossover', 'stoch_rsi_bearish_crossover', 'stoch_rsi_trend', 'stoch_rsi_change_of_trend']
features_hma = ['hma200_trend', 'hma200_change_of_trend', 'hma200_above_price']
features_macd = ['macd_bullish_crossover', 'macd_bearish_crossover', 'macd_positive', 'macd_negative', 'macd_bullish_crossover_positive', 'macd_bearish_crossover_negative', 'macd_trend', 'macd_change_of_trend']

features_base = ['timestamp', 'close']
features_stochrsi = ['stoch_rsi_overbought', 'stoch_rsi_oversold', 'stoch_rsi_bullish_crossover', 'stoch_rsi_bearish_crossover', 'stoch_rsi_trend']
features_hma = ['hma200_trend', 'hma200_above_price']
features_macd = ['macd_bullish_crossover', 'macd_bearish_crossover', 'macd_positive', 'macd_negative', 'macd_bullish_crossover_positive', 'macd_bearish_crossover_negative', 'macd_trend']

columns = features_base + features_stochrsi + features_hma + features_macd



df = pd.read_csv(f"{folder}/{file}")

len(df)

3680586

In [44]:
def get_best_group(df, features, period, sort='mean', max=5):
    grouped_df = df.groupby(features)[f'close_p_in_{period}'].agg(['count', 'mean', 'min', 'max'])
    grouped_df = grouped_df.sort_values(by=sort, ascending=False)
    return grouped_df.head(max)


def convert_grouped_to_dataframe(grouped_df):
    df = grouped_df.reset_index()
    return df

# import pandas as pd

def append_dataframes(df1, df2):
    # Concatenate the dataframes
    df = pd.concat([df1, df2], ignore_index=True)
    return df

we want to work on the history 1m from bitcoin (gemini)

Dataframe must be indexed by timestamp

In [45]:
df = df[columns]
df = df.set_index('timestamp')

df.tail(5)

,close,stoch_rsi_overbought,stoch_rsi_oversold,stoch_rsi_bullish_crossover,stoch_rsi_bearish_crossover,stoch_rsi_trend,hma200_trend,hma200_above_price,macd_bullish_crossover,macd_bearish_crossover,macd_positive,macd_negative,macd_bullish_crossover_positive,macd_bearish_crossover_negative,macd_trend
timestamp,,,,,,,,,,,,,,,
2024-01-01 00:04:00,42366.62,False,True,False,False,Downtrend,Downtrend,True,False,False,False,True,False,False,Downtrend
2024-01-01 00:03:00,42355.02,False,True,False,False,Downtrend,Downtrend,True,False,False,False,True,False,False,Downtrend
2024-01-01 00:02:00,42317.89,False,True,False,False,Downtrend,Downtrend,True,False,False,False,True,False,False,Downtrend
2024-01-01 00:01:00,42300.77,False,True,False,False,Not trending,Downtrend,True,False,False,False,True,False,False,Downtrend
2024-01-01 00:00:00,42278.18,False,True,False,False,Not trending,Downtrend,True,False,False,False,True,False,False,Downtrend


Now we calculate the result for different number of intervals

In [46]:
# adding shifted periods results
features_close = [f'close_in_{period}' for period in periods]
features_close_perc = [f'close_p_in_{period}' for period in periods]
for period in periods:
    # column_name = f'close_in_{period}'
    # df[column_name] = df['close'].shift(-period)
    column_perc = f'close_p_in_{period}'
    df[column_perc] = ((df['close'].shift(-period) - df['close']) / df['close']) * 100
    
df.to_csv(f"{folder_lab1}/{file}_periods.csv")
df.columns

Index(['close', 'stoch_rsi_overbought', 'stoch_rsi_oversold',
       'stoch_rsi_bullish_crossover', 'stoch_rsi_bearish_crossover',
       'stoch_rsi_trend', 'hma200_trend', 'hma200_above_price',
       'macd_bullish_crossover', 'macd_bearish_crossover', 'macd_positive',
       'macd_negative', 'macd_bullish_crossover_positive',
       'macd_bearish_crossover_negative', 'macd_trend', 'close_p_in_10'],
      dtype='object')

we classify the features based on the result for the given number of intervals

In [47]:
# Create a column variable that is the combination of all the features_ lists
all_features = features_stochrsi + features_hma + features_macd



# removing likely not valid rows
features = all_features + [f'close_p_in_{period}']
new_df = df[features].iloc[50:-period]

len(new_df)


3680526

In [8]:
# import pandas as pd

# # Assuming `df` is your DataFrame with OHLC data, features, and a 'future_return' column 
# # that represents the percent increase after N periods.

# # Calculate Pearson correlation coefficients
# correlation_matrix = new_df.corr()
# performance_correlations = correlation_matrix[f'close_p_in_{period}'].sort_values(ascending=False)

# print(performance_correlations)

ValueError: could not convert string to float: 'Uptrend'

1. Chi-Square Test for Independence
The Chi-Square test is useful for testing the relationship between two categorical variables. It can tell you whether the distribution of sample categorical data matches an expected distribution.

Use Case: To determine if there is a significant association between two categorical features.

In [48]:
import numpy as np
from scipy.stats import chi2_contingency

# Assuming df is your DataFrame and 'feature' and 'target' are your categorical columns

results = []

new_df['target'] = np.where(new_df[f'close_p_in_{period}'] > 0, 'Increase', 'Decrease')
for feature in all_features:
    contingency_table = pd.crosstab(new_df[feature], new_df['target'])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    results.append({'feature': feature, 'chi2': chi2, 'p-value': p})

# Sort the results by chi2 in descending order
results = sorted(results, key=lambda x: x['chi2'], reverse=True)

# Display the sorted results
for result in results:
    print(f"Feature: {result['feature']}, Chi-Square Value: {result['chi2']}, P-Value: {result['p-value']}")



Feature: macd_trend, Chi-Square Value: 8827.558450215423, P-Value: 0.0
Feature: macd_positive, Chi-Square Value: 4604.647794152068, P-Value: 0.0
Feature: macd_negative, Chi-Square Value: 3686.688225578986, P-Value: 0.0
Feature: stoch_rsi_trend, Chi-Square Value: 3008.744360295424, P-Value: 0.0
Feature: hma200_above_price, Chi-Square Value: 2702.514769918251, P-Value: 0.0
Feature: hma200_trend, Chi-Square Value: 2670.7635168501283, P-Value: 0.0
Feature: stoch_rsi_overbought, Chi-Square Value: 353.37716785636394, P-Value: 7.79333591557036e-79
Feature: macd_bullish_crossover_positive, Chi-Square Value: 338.2676233748225, P-Value: 1.5210875127704433e-75
Feature: stoch_rsi_bearish_crossover, Chi-Square Value: 275.87698518162, P-Value: 5.9442345302848744e-62
Feature: stoch_rsi_bullish_crossover, Chi-Square Value: 194.21463409608418, P-Value: 3.8231299613468623e-44
Feature: macd_bullish_crossover, Chi-Square Value: 136.84391773191498, P-Value: 1.3044591354578955e-31
Feature: macd_bearish_cros

In [49]:
cluster1 = ['hma200_trend','hma200_above_price','macd_positive','macd_negative']
get_best_group(new_df, cluster1, period)

count  \
hma200_trend hma200_above_price macd_positive macd_negative            
Uptrend      True               False         True             63685   
Not trending True               False         False               76   
Uptrend      False              False         False           145639   
             True               True          False            49771   
Downtrend    True               False         True           1285166   

                                                                 mean  \
hma200_trend hma200_above_price macd_positive macd_negative             
Uptrend      True               False         True           0.055899   
Not trending True               False         False          0.031140   
Uptrend      False              False         False          0.018278   
             True               True          False          0.014307   
Downtrend    True               False         True           0.013197   

                                                                   min  \
hma200_trend hma200_above_price macd_positive macd_negative              
Uptrend      True               False         True           -5.660377   
Not trending True               False         False          -0.710542   
Uptrend      False              False         False          -6.761538   
             True               True          False         -12.131747   
Downtrend    True               False         True          -43.034994   

                                                                    max  
hma200_trend hma200_above_price macd_positive macd_negative              
Uptrend      True               False         True           686.591156  
Not trending True               False         False            1.950809  
Uptrend      False              False         False          686.885872  
             True               True          False           15.297016  
Downtrend    True               False         True            63.280796

In [52]:
cluster2 = cluster1 + ['macd_trend','stoch_rsi_trend']
# cluster3 = cluster2 + ['stoch_rsi_overbought','macd_bullish_crossover_positive']
get_best_group(new_df, cluster2, period)

count  \
hma200_trend hma200_above_price macd_positive macd_negative macd_trend   stoch_rsi_trend          
Not trending True               True          False         Uptrend      Not trending         3   
Uptrend      True               True          False         Not trending Uptrend              1   
Downtrend    False              False         False         Downtrend    Not trending         8   
                                              True          Downtrend    Downtrend         3129   
Uptrend      True               False         False         Not trending Downtrend            3   

                                                                                              mean  \
hma200_trend hma200_above_price macd_positive macd_negative macd_trend   stoch_rsi_trend             
Not trending True               True          False         Uptrend      Not trending     0.382119   
Uptrend      True               True          False         Not trending Uptrend          0.215709   
Downtrend    False              False         False         Downtrend    Not trending     0.215578   
                                              True          Downtrend    Downtrend        0.209275   
Uptrend      True               False         False         Not trending Downtrend        0.159682   

                                                                                               min  \
hma200_trend hma200_above_price macd_positive macd_negative macd_trend   stoch_rsi_trend             
Not trending True               True          False         Uptrend      Not trending     0.381782   
Uptrend      True               True          False         Not trending Uptrend          0.215709   
Downtrend    False              False         False         Downtrend    Not trending    -0.137295   
                                              True          Downtrend    Downtrend       -4.552204   
Uptrend      True               False         False         Not trending Downtrend       -0.102834   

                                                                                                 max  
hma200_trend hma200_above_price macd_positive macd_negative macd_trend   stoch_rsi_trend              
Not trending True               True          False         Uptrend      Not trending       0.382287  
Uptrend      True               True          False         Not trending Uptrend            0.215709  
Downtrend    False              False         False         Downtrend    Not trending       0.730106  
                                              True          Downtrend    Downtrend        277.778462  
Uptrend      True               False         False         Not trending Downtrend          0.521446

In [ ]:

get_best_group(new_df, features_stochrsi, period)

In [ ]:

df_result = get_best_group(new_df, all_features, period, max=10, sort='mean')
df_result

In [ ]:

get_best_group(new_df, features_macd, period)

In [ ]:

get_best_group(new_df, features_stochrsi+features_macd, period)

In [ ]:
folder_lab1 = "./Data/lab1"
features_to_test = features_stochrsi
for period in periods:
    print(f"Period: {period}")
    features = all_features + [f'close_p_in_{period}']
    new_df = df[features].iloc[50:-period]
    result = get_best_group(new_df, features_to_test, period, 'count')
    result.to_csv(f"{folder_lab1}/{file}_stochrsi_{period}.csv")
    print(result)
    
    # print(get_best_group(new_df, features_hma, period))
    # print(get_best_group(new_df, features_macd, period, 'count'))
    # print(get_best_group(new_df, features_stochrsi+features_macd, period))
    print("")

i want to find the best combination of features, maximising the results

now we find the combo of features that are performing better

In [ ]:
import itertools

folder_lab1 = "./Data/lab1"
file = "gemini_1m_features.csv"
# features_to_test = features_stochrsi 
features_to_test = features_stochrsi 

combinations_stochrsi = []
for r in range(1, len(features_stochrsi) + 1):
    combinations_stochrsi.extend(itertools.combinations(features_stochrsi, r))
print(len(combinations_stochrsi)) 

combinations_hma = []
for r in range(1, len(features_hma) + 1):
    combinations_hma.extend(itertools.combinations(features_hma, r))
print(len(combinations_hma)) 

combinations_macd = []
for r in range(1, len(features_macd) + 1):
    combinations_macd.extend(itertools.combinations(features_macd, r))
print(len(combinations_macd)) 

cartesian_product = list(itertools.product(combinations_stochrsi, combinations_hma, combinations_macd))
print(len(cartesian_product))


In [ ]:

folder_lab1 = "./Data/lab1"
file = "gemini_1m_features.csv"

features_to_test = features_stochrsi 



for period in periods:
    analysis_df = pd.DataFrame()
    print(f"Period: {period}")
    features = all_features + [f'close_p_in_{period}']
    new_df = df[features].iloc[50:-period]
    for combination in combinations_stochrsi:
        list_of_combination = list(combination)
        if len(list_of_combination) < 3:
            continue
        result = get_best_group(new_df, list_of_combination, period, 'count')
        tmp_df = convert_grouped_to_dataframe(result)
        tmp_df['period'] = period
        analysis_df = append_dataframes(tmp_df,analysis_df)
        # print(result)
        print("")
    analysis_df.to_csv(f"{folder_lab1}/{file}_stochrsi_analysis_{period}.csv")

In [ ]:

analysis_df = pd.read_csv(f"{folder_lab1}/{file}_stochrsi_analysis_{period}.csv")

# Filter the dataframe to include only rows where the count is greater than 1
filtered_df = analysis_df[analysis_df['count'] > 1]

# I want to minimise the losses
# filtered_df = filtered_df[filtered_df['mean'] > filtered_df['min'].abs()]

# Sort the dataframe by mean in descending order and by the difference between min and max in ascending order
sorted_df = filtered_df.sort_values(by=['mean', 'max', 'min'], ascending=[False, True, True])

sorted_df.head(5)
